In [2]:
import pandas as pd

In [3]:
commodities =["AMETHYSTS","STARFRUIT"]

In [4]:
trades = pd.read_csv("./data/round1/trades_round_1_day_-1_nn.csv", sep=";")

In [5]:
all_prices = pd.read_csv("./data/round1/prices_round_1_day_-1.csv", sep=";", index_col=1)
prices = {}
for commo in commodities: 
    prices[commo] = all_prices[all_prices["product"]==commo].copy()

In [42]:
import numpy as np
market_prices = {}
for commo in ["STARFRUIT"]: 
    trades_commo = trades[trades["symbol"]==commo].copy()
    trades_commo["volume"] = trades_commo["price"]*trades_commo["quantity"]
    market_prices[commo] = trades_commo.groupby("timestamp")["volume"].sum()/trades_commo.groupby("timestamp")["quantity"].sum()
    trades_commo.set_index("timestamp",inplace=True)
    prices[commo]["market_price"] = market_prices[commo]
    
    dirs = []
    for i in range(len(trades_commo)): 
        row = trades_commo.iloc[i]
        price = prices[commo].loc[row.name]
        if price["bid_price_1"] == row["price"]:
            dirs.append(-1)
        elif price["ask_price_1"] == row["price"]: 
            dirs.append(1)
        else:
            dirs.append(np.nan)
    
    trades_commo["dir"] = dirs

In [43]:
trades_commo["inventory_change"] = -trades_commo["dir"]*trades_commo["quantity"]

In [44]:
trades_commo["inventory"] = trades_commo["inventory_change"].cumsum()

In [45]:
trades_commo["inventory_value"] = trades_commo["inventory"]*trades_commo["price"]

In [46]:
trades_commo["flows"] = -trades_commo["dir"]*trades_commo["quantity"]*trades_commo["price"]

In [47]:
px.line(trades_commo["inventory_value"]-trades_commo["flows"].cumsum())

In [120]:
prices["AMETHYSTS"].ask_price_1.diff()

timestamp
0         NaN
100       0.0
200       1.0
300      -1.0
400       0.0
         ... 
999500    0.0
999600    0.0
999700    1.0
999800   -3.0
999900    2.0
Name: ask_price_1, Length: 10000, dtype: float64

In [119]:
import plotly_express as px
px.line(((prices["STARFRUIT"]["mid_price"]-prices["STARFRUIT"]["market_price"].ffill())/prices["STARFRUIT"]["market_price"].ffill()).rolling(1000).mean())

In [118]:
px.line((prices["STARFRUIT"]["mid_price"]).pct_change(1000))

In [108]:
from statsmodels.tsa.stattools import adfuller

In [114]:
adftest = adfuller(prices["AMETHYSTS"]["mid_price"].pct_change().dropna())
print("ADF Test Results")
print("Null Hypothesis: The series has a unit root (non-stationary)")
print("ADF-Statistic:", adftest[0])
print("P-Value:", adftest[1])
print("Number of lags:", adftest[2])
print("Number of observations:", adftest[3])
print("Critical Values:", adftest[4])
print("Note: If P-Value is smaller than 0.05, we reject the null hypothesis and the series is stationary")

ADF Test Results
Null Hypothesis: The series has a unit root (non-stationary)
ADF-Statistic: -27.882757587835567
P-Value: 0.0
Number of lags: 38
Number of observations: 9960
Critical Values: {'1%': -3.4310067255162737, '5%': -2.86183023348434, '10%': -2.5669244861474008}
Note: If P-Value is smaller than 0.05, we reject the null hypothesis and the series is stationary
